NEW code for search algo

In [10]:
# Import the 'os' module for operating system-related functionalities
import os

# Import the 'copy' module for creating shallow and deep copies of objects
import copy

# Import the 'csv' module for reading and writing CSV files
import csv

# Import the 'graph' module (assuming it's a custom module for graph-related operations)
import graph

# Import the 'heapq' module for heap queue algorithm operations
import heapq

# Import the 'CsvWriter' class from the custom 'helper_file' module
from helper_file import CsvWriter

In [11]:
class problem:
    def __init__(self, pnr, fr, to, sch, cf):
        self.id = pnr
        self.startnode = fr
        self.endnode = to
        self.schedule = sch
        self.costfunction = cf

    @staticmethod
    # cost vector
    def cost(linear: str) -> []:
        costs_value = [0, 0, 0]
        costs = ["stops", "traveltime", "price"]

        if linear in costs:
            i = costs.index(linear)
            costs_value[i] = 1

        if linear.split()[0] == "arrivaltime":
            costs_value.append(1)
        else:
            costs_value.append(0)

        return costs_value

    @staticmethod
    def time_to_sec(times: []) -> int:
        hours = times[0].strip().strip('\'')
        minutes = times[1].strip().strip('\'')
        seconds = times[2].strip().strip('\'')

        return (((int(hours) * 60) + int(minutes)) * 60) + int(seconds)

    @staticmethod
    def duration_counter(arrivaltime: str, departuretime: str):
        arr_l = arrivaltime.split(':')
        dep_l = departuretime.split(':')
        day_lowest = 0  # in sec
        day_heighest = 86400  # in sec

        arr_time_sec = problem.time_to_sec(arr_l)
        dep_time_sec = problem.time_to_sec(dep_l)

        if arr_time_sec < dep_time_sec:
            duration = (day_heighest - dep_time_sec) + (arr_time_sec - day_lowest)
            return duration
        else:
            return arr_time_sec - dep_time_sec

    @staticmethod
    def day_change(arrivaltime: str, departuretime: str):
        arr_l = arrivaltime.split(':')
        dep_l = departuretime.split(':')


        arr_time_sec = problem.time_to_sec(arr_l)
        dep_time_sec = problem.time_to_sec(dep_l)

        if arr_time_sec < dep_time_sec:
            return 1 
        else:
            return 0

    @staticmethod
    def arrivaltime (day: int, arrivaltime: str):
        if day == 0:
            return arrivaltime.strip().strip('\'')
        elif day < 10:
            return '0' + str(day) + ':' + arrivaltime.strip().strip('\'')
        else:
            return str(day) + ':' + arrivaltime.strip().strip('\'')

In [12]:
class Solution:
    def __init__(self, pn, time, stnode):
        # Initialize the solution object with provided parameters
        self.pnr = pn  # train number
        self.currentnode = stnode  # Current node (station)
        self.currentline = ' '  # Current line
        self.prevline = ' '  # Previous line
        self.route = {}  # Dictionary to store the route
        self.rnr = 0  # Route number
        self.cost = 0  # Cost of the route
        self.visited = {stnode}  # Set of visited nodes (stations)
        self.day = 0  # Day of the route
        self.currenttime = time  # Current time in the route
        self.done = False  # Flag indicating whether the route is completed


In [13]:
class dnode:
    def __init__(self, id, lin, nlin, cos, pre, da, arrival):
        # Initialize a DNode object with the given parameters
        self.id = id  # Unique identifier for the node (train)
        self.line = lin  # Current line associated with the node
        self.nextline = nlin  # Next line after this node
        self.cost = cos  # Cost of reaching this node
        self.prev = pre  # Previous node in the route
        self.day = da  # Day of the route when this node is encountered
        self.arrivaltime = arrival  # Arrival time at this node
        self.lines = {}  # Dictionary to store additional information about lines related to this node

In [14]:
class dline:
    def __init__(self, lid, co, sid, nid, sol):
        # Initialize a dline object with the given parameters
        self.id = lid  # Unique identifier for the line
        self.cost = co  # Cost associated with the line
        self.station = sid  # Current station associated with the line
        self.nextStation = nid  # Next station after this line
        self.solution = sol  # Solution related to this line

    def __lt__(self, other):
        # Define less-than comparison based on the cost attribute
        return self.cost < other.cost

In [15]:
def dsearch(problem, network):

    'initialisation'
    # extract cost multipliers
    costs = problem.cost(problem.costfunction)

    stops = costs[0]
    duration = costs[1] # to be changed
    price = costs[2]
    arrivaltime = costs[3]

    # initialise lists, dicts and heap
    visitednodes = {}
    routelines = []

    workedlines = []

    lineheap = []

    line = ' '
    cost = 0
    prev = ' '

    # initialise connections from the startnode
    # we make special abbrivated dijkstra nodes for the dijkstra network
    # in case a node a several possible incoming routes with the same cost, we save several possible solutions


    if arrivaltime == 1:

        time = problem.costfunction.split()[1]

        for x in network[problem.startnode].schedule:
            w = network[problem.startnode].schedule[x]
            initsolution = solution(problem.id, time, problem.startnode)
            visitednodes[problem.startnode] = dnode(problem.startnode, line, ' ', cost, prev, 0, time)
            
            if w.nextStation != ' ':
                worksolution = copy.deepcopy(initsolution)

                day = problem.day_change(w.departure, time)

                # cost calculation block
                linecost = visitednodes[problem.startnode].cost + stops * 1 + price

                'time cost calculations'
                linecost += duration*problem.duration_counter(network[w.nextStation].schedule[w.id].arrival, w.departure)

                linecost += arrivaltime*problem.duration_counter(w.departure, time)
                linecost += arrivaltime*problem.duration_counter(network[w.nextStation].schedule[w.id].arrival, w.departure)


                # we make special dijkstra lines, like with the nodes
                # in each line, a solution object is saved. Since we look at each line at most once, this allows us to
                # build the solution on the fly
                worksolution.day = day
                worksolution.cost = linecost
                worksolution.currentline = w.id
                worksolution.currenttime = network[w.nextStation].schedule[w.id].arrival
                worksolution.visited.add(problem.startnode)
                worksolution.route[0] = (w.id, [w.stop])
                worksolution.route[0][1].append(network[w.nextStation].schedule[w.id].stop)
                
                newline = dline(w.id, linecost, w.currentStation, w.nextStation, worksolution)

                visitednodes[problem.startnode].lines[w.id] = newline

                heapq.heappush(lineheap, newline)

                network[problem.startnode].schedule[x].cost = linecost
                workedlines.append(newline)
    
    else:
        for x in network[problem.startnode].schedule:
            w = network[problem.startnode].schedule[x]
            initsolution = solution(problem.id, w.arrival, problem.startnode)
            visitednodes[problem.startnode] = dnode(problem.startnode, line, ' ', cost, prev, 0, w.arrival)
            
            if w.nextStation != ' ':
                worksolution = copy.deepcopy(initsolution)

                if w.stop == "'1'":
                    w.arrival = w.departure

                day = problem.day_change(w.departure, w.arrival)

                # cost calculation block
                linecost = visitednodes[problem.startnode].cost + stops * 1 + price

                'time cost calculations'
                linecost += duration*problem.duration_counter(network[w.nextStation].schedule[w.id].arrival, w.departure)

                linecost += arrivaltime*problem.duration_counter(w.departure, w.arrival)
                linecost += arrivaltime*problem.duration_counter(network[w.nextStation].schedule[w.id].arrival, w.departure)

                # we make special dijkstra lines, like with the nodes
                # in each line, a solution object is saved. Since we look at each line at most once, this allows us to
                # build the solution on the fly
                worksolution.day = day
                worksolution.cost = linecost
                worksolution.currentline = w.id
                worksolution.currenttime = network[w.nextStation].schedule[w.id].arrival
                worksolution.visited.add(problem.startnode)
                worksolution.route[0] = (w.id, [w.stop])
                worksolution.route[0][1].append(network[w.nextStation].schedule[w.id].stop)

                newline = dline(w.id, linecost, w.currentStation, w.nextStation, worksolution)

                visitednodes[problem.startnode].lines[w.id] = newline

                heapq.heappush(lineheap, newline)

                network[problem.startnode].schedule[x].cost = linecost
                workedlines.append(newline)
    

    while(True):

        'Dijkstra loop'

        # pop a line with lowest cost from the heap, and get the line from the actual network
        workline = heapq.heappop(lineheap)
        netline = network[workline.station].schedule[workline.id]

        if netline.nextStation == problem.endnode:
            'if the line reaches the goal, we can directly use the solution saved in the workline object'

            solved = workline.solution

            # if we are only interested in the arrival time, we change the cost to the right format
            if arrivaltime == 1 and duration == 0 and price == 0 and stops == 0:
                solved.cost = problem.arrivaltime(solved.day, solved.currenttime)
                

            elif arrivaltime == 0 and duration == 0 and price == 1 and stops == 0:
                # solved.cost += -1 # subtracting the initial cost
                solved.cost += solved.day # adding overall cost
                

            else:
                # print("Solution found for:  " + str(solved.day)+" " + problem.id + ", Cost: " + str(solved.cost))
                pass

            for x in workedlines:
                'reset the cost of all network schedule objects, just to be save'
                network[x.station].schedule[x.id].cost = 0

            return solved

        elif netline.nextStation not in workline.solution.visited:

            'Dijkstra update if a specific line between to nodes in the network has not been used yet'
            # set the line in the network to visited, so that we don't use it again
            #network[workline.station].schedule[workline.id].visited = True

            for x in network[netline.nextStation].schedule:

                # extract all lines to consider from the schedule of the next station
                w = network[netline.nextStation].schedule[x]

                if w.nextStation != ' ':

                    # calculation of the simple costs
                    newcost = workline.cost + stops  #+ price * w.distance + 1 * w.distance
                
                    if w.id != workline.id:
                        newcost +=  price 

                    'time management'
                    # calculates duration (on train) and travel time (incl. waiting times)

                    traveltime = problem.duration_counter(network[w.nextStation].schedule[x].arrival, w.departure)
                    traveltime += problem.duration_counter(w.departure, network[netline.nextStation].schedule[workline.id].arrival)

                    newcost += duration * traveltime
                    newcost += arrivaltime * traveltime

                    if newcost < w.cost or w.cost == 0:

                        worksolution = copy.deepcopy(workline.solution)
                        # update the line object in the network with the lowest cost
                        network[netline.nextStation].schedule[x].cost = newcost
                        # counting days in case of waiting and traveling time having day price
                        newday = workline.solution.day
                        newday += problem.day_change(w.departure, network[netline.nextStation].schedule[workline.id].arrival)
                        newday += problem.day_change(network[w.nextStation].schedule[w.id].arrival, w.departure)

                        # copy and update the solution and add it to a new line object, which is pushed to the heap
                        worksolution.visited.add(w.currentStation)
                        worksolution.currenttime = network[w.nextStation].schedule[w.id].arrival
                        worksolution.day = newday
                        worksolution.cost = newcost

                        if worksolution.currentline != w.id:
                            worksolution.rnr += 1
                            worksolution.route[worksolution.rnr] = (w.id, [w.stop])
                            worksolution.route[worksolution.rnr][1].append(network[w.nextStation].schedule[w.id].stop)
                        else:
                            worksolution.route[worksolution.rnr][1].append(network[w.nextStation].schedule[w.id].stop)

                        worksolution.prevline = worksolution.currentline
                        worksolution.currentline = w.id

                        newline = dline(w.id, newcost, w.currentStation, w.nextStation, worksolution)

                        'node and line update'
                        if netline.nextStation not in visitednodes:
                            # if we haven't visited a station yet, or come across a lower cost, we simply reinit it
                            newnode = dnode(netline.nextStation, workline.id, newline, newcost, netline.currentStation, newday, network[netline.nextStation].schedule[netline.id].arrival)
                            visitednodes[netline.nextStation] = newnode

                        visitednodes[netline.nextStation].lines[workline.id] = newline

                        heapq.heappush(lineheap, newline)
                        workedlines.append(newline)

In [16]:
def main():
    network1 = graph.Graph("schedule.csv")
    network2 =  graph.Graph("mini-schedule.csv")
    
    # Check if the file exists
    file_path = "solutions.csv"

    if os.path.exists(file_path):
    # Delete previous solution file
        os.remove(file_path)


    problems = []

    file = open('probleme1.csv')
    reader = csv.reader(file)

    #extract the header from the problem file
    header = []
    header = next(reader)

    rows = []
    for row in reader:
        rows.append(row)

    file.close()

    for x in rows:
        # add all problems from `problems.csv`
        problems.append(problem(x[0], x[1], x[2], x[3], x[4]))

    for x in problems:
        # we only have to take the first solution from the search
        # write one solution at a time on `solutions.csv`
        if x.schedule == "schedule.csv":
            solution = dsearch(x, network1)
        else:
            solution = dsearch(x, network2)

        CsvWriter.WriteCsvFile1(solution)

    print("Execution complete. Please check solutions.csv for answers!")

In [17]:
main()

Execution complete. Please check solutions.csv for answers!
